In [16]:
import pandas as pd

In [17]:
DATA_PATH = "../data/Telco_customer_churn.xlsx"
df = pd.read_excel(DATA_PATH)

df.shape

(7043, 33)

In [18]:
df.head(5)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [19]:
# STEP 1 — Define target and features
X = df.drop(columns=["Churn Value"])
y = df["Churn Value"]

print("X shape:", X.shape)
print("y shape:", y.shape)

print("\nTarget distribution (counts):")
print(y.value_counts())

print("\nTarget distribution (percent):")
print((y.value_counts(normalize=True) * 100).round(2))


X shape: (7043, 32)
y shape: (7043,)

Target distribution (counts):
Churn Value
0    5174
1    1869
Name: count, dtype: int64

Target distribution (percent):
Churn Value
0    73.46
1    26.54
Name: proportion, dtype: float64


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

print("\ny_train distribution (percent):")
print((y_train.value_counts(normalize=True) * 100).round(2))

print("\ny_test distribution (percent):")
print((y_test.value_counts(normalize=True) * 100).round(2))


X_train shape: (5634, 32)
X_test shape: (1409, 32)

y_train distribution (percent):
Churn Value
0    73.46
1    26.54
Name: proportion, dtype: float64

y_test distribution (percent):
Churn Value
0    73.46
1    26.54
Name: proportion, dtype: float64


In [21]:
# STEP 3 — Identify feature types on X_train

num_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

print("Numerical features:")
print(num_features)

print("\nCategorical features:")
print(cat_features)

print("\nNumber of numerical features:", len(num_features))
print("Number of categorical features:", len(cat_features))


Numerical features:
['Count', 'Zip Code', 'Latitude', 'Longitude', 'Tenure Months', 'Monthly Charges', 'Churn Score', 'CLTV']

Categorical features:
['CustomerID', 'Country', 'State', 'City', 'Lat Long', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Total Charges', 'Churn Label', 'Churn Reason']

Number of numerical features: 8
Number of categorical features: 24


In [22]:
# STEP 4 — Drop leakage and non-informative columns

columns_to_drop = [
    "CustomerID",
    "Country",
    "State",
    "City",
    "Lat Long",
    "Zip Code",
    "Latitude",
    "Longitude",
    "Count",
    "Churn Label",
    "Churn Reason",
    "Churn Score"
]

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

print("X_train shape after drop:", X_train.shape)
print("X_test shape after drop:", X_test.shape)


X_train shape after drop: (5634, 20)
X_test shape after drop: (1409, 20)


Columns that could cause data leakage, act as identifiers, or provide no predictive value were removed from the dataset.
This step ensures that the model learns only from relevant customer information and avoids biased predictions.

In [ ]:
# Fix Total Charges data type

# Convert Total Charges to numeric (invalid values become NaN)
X_train["Total Charges"] = pd.to_numeric(
    X_train["Total Charges"], errors="coerce"
)

X_test["Total Charges"] = pd.to_numeric(
    X_test["Total Charges"], errors="coerce"
)

# Check result
print("X_train Total Charges dtype:", X_train["Total Charges"].dtype)
print("X_test Total Charges dtype:", X_test["Total Charges"].dtype)

print("\nMissing values in Total Charges (train):")
print(X_train["Total Charges"].isnull().sum())

print("\nMissing values in Total Charges (test):")
print(X_test["Total Charges"].isnull().sum())


X_train Total Charges dtype: float64
X_test Total Charges dtype: float64

Missing values in Total Charges (train):
8

Missing values in Total Charges (test):
3


The Total Charges variable was initially stored as a text field due to the presence of empty string values.
These empty values correspond to customers with very short tenure.
The column was safely converted to a numerical format by coercing invalid values to missing values, which were handled later during the imputation step.

In [24]:
# Check rows where Total Charges is missing
missing_tc_train = X_train[X_train["Total Charges"].isna()]

print("Number of missing Total Charges (train):", missing_tc_train.shape[0])

# Look at tenure for those rows
missing_tc_train[["Tenure Months", "Monthly Charges", "Total Charges"]]


Number of missing Total Charges (train): 8


,Tenure Months,Monthly Charges,Total Charges
6772,0,73.35,NaN
5104,0,20.00,NaN
4687,0,25.35,NaN
2234,0,52.55,NaN
2667,0,25.75,NaN
2856,0,56.05,NaN
6840,0,61.90,NaN
4331,0,19.85,NaN


In [25]:

num_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X_train.select_dtypes(include=["object"]).columns.tolist()

print("Final numerical features:")
print(num_features)

print("\nFinal categorical features:")
print(cat_features)

print("\nNumber of numerical features:", len(num_features))
print("Number of categorical features:", len(cat_features))


Final numerical features:
['Tenure Months', 'Monthly Charges', 'Total Charges', 'CLTV']

Final categorical features:
['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method']

Number of numerical features: 4
Number of categorical features: 16


After cleaning the dataset, numerical and categorical variables were re-identified based on their actual meaning rather than their raw data types.
This final feature selection step ensured that appropriate preprocessing techniques were applied to each variable type

In [ ]:
# STEP 5 — Remove outliers using IQR method

import numpy as np

def remove_outliers_iqr(df, numerical_cols, factor=1.5):
    """
    Remove outliers using the Interquartile Range (IQR) method.
    
    Parameters:
    - df: DataFrame to clean
    - numerical_cols: List of numerical column names
    - factor: IQR multiplier (default 1.5, use 3.0 for more conservative removal)
    
    Returns:
    - df_clean: DataFrame with outliers removed
    - outlier_mask: Boolean mask indicating which rows were removed
    """
    df_clean = df.copy()
    outlier_mask = np.zeros(len(df_clean), dtype=bool)
    
    for col in numerical_cols:
        if col not in df_clean.columns:
            continue
            
        # Skip if column has too many missing values
        if df_clean[col].isna().sum() > len(df_clean) * 0.5:
            continue
            
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        
        # Mark outliers for this column
        col_outliers = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outlier_mask = outlier_mask | col_outliers
        
        print(f"\n{col}:")
        print(f"  Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
        print(f"  Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Outliers found: {col_outliers.sum()}")
    
    print(f"\nTotal rows with outliers: {outlier_mask.sum()}")
    print(f"Percentage of data removed: {(outlier_mask.sum() / len(df_clean) * 100):.2f}%")
    
    df_clean = df_clean[~outlier_mask]
    return df_clean, outlier_mask

# Store original shapes
print("BEFORE outlier removal:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Remove outliers from training set
print("\n" + "="*50)
print("Removing outliers from TRAINING set:")
print("="*50)
X_train_clean, train_outlier_mask = remove_outliers_iqr(
    X_train, 
    num_features, 
    factor=1.5  # Adjust this: 1.5 (standard) or 3.0 (conservative)
)

# Remove corresponding rows from y_train
y_train_clean = y_train[~train_outlier_mask]

# Apply same outlier criteria to test set (using bounds from training set)
print("\n" + "="*50)
print("Applying outlier criteria to TEST set:")
print("="*50)

# Calculate bounds from training set
outlier_bounds = {}
for col in num_features:
    if col not in X_train.columns:
        continue
    if X_train[col].isna().sum() > len(X_train) * 0.5:
        continue
    
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)
    IQR = Q3 - Q1
    factor = 1.5
    
    outlier_bounds[col] = {
        'lower': Q1 - factor * IQR,
        'upper': Q3 + factor * IQR
    }

# Apply bounds to test set
test_outlier_mask = np.zeros(len(X_test), dtype=bool)
for col in num_features:
    if col not in X_test.columns or col not in outlier_bounds:
        continue
    
    bounds = outlier_bounds[col]
    col_outliers = (X_test[col] < bounds['lower']) | (X_test[col] > bounds['upper'])
    test_outlier_mask = test_outlier_mask | col_outliers
    
    print(f"\n{col}:")
    print(f"  Bounds: [{bounds['lower']:.2f}, {bounds['upper']:.2f}]")
    print(f"  Outliers found: {col_outliers.sum()}")

print(f"\nTotal rows with outliers in test set: {test_outlier_mask.sum()}")
print(f"Percentage of test data removed: {(test_outlier_mask.sum() / len(X_test) * 100):.2f}%")

X_test_clean = X_test[~test_outlier_mask]
y_test_clean = y_test[~test_outlier_mask]

# Update the datasets
X_train = X_train_clean
X_test = X_test_clean
y_train = y_train_clean
y_test = y_test_clean

print("\n" + "="*50)
print("AFTER outlier removal:")
print("="*50)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print("\nTarget distribution after outlier removal (train):")
print((y_train.value_counts(normalize=True) * 100).round(2))

Outliers were removed using the Interquartile Range (IQR) method, which identifies values outside the range [Q1 - 1.5×IQR, Q3 + 1.5×IQR] for each numerical feature.
This method is robust and helps prevent extreme values from skewing the model's learning process.
The same outlier thresholds calculated from the training set were applied to the test set to ensure consistency and prevent data leakage.

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


In [27]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [28]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ],
    remainder="drop"
)


A column transformer was used to apply different preprocessing steps to numerical and categorical variables.
Numerical features were imputed using the median and standardized, while categorical features were imputed using the most frequent value and one-hot encoded.

In [29]:

X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

print("X_train_prepared shape:", X_train_prepared.shape)
print("X_test_prepared shape:", X_test_prepared.shape)


X_train_prepared shape: (5634, 47)
X_test_prepared shape: (1409, 47)
